In [9]:
import pandas as pd
from pandasql import sqldf
import pyparsing as pyp
from os import path
import re
import string

# TODO Solve this problem with the relative paths.
DATA_IN = '../../data/clean/Jefferson_County_KY_Street_Intersections.csv'
assert path.exists(DATA_IN)

df = pd.read_csv(DATA_IN, index_col=0)
df.head(5)

,intid,main_street,cross_street,main_suffix,cross_suffix,cross_prefix,main_prefix,longitude,latitude
OBJECTID,,,,,,,,,
1,154647662,REHL,REHL,RD,CT,W,,-85.510444,38.205887
2,254646551,REHL,TUCKER STATION,RD,RD,,,-85.528169,38.200376
3,354646551,REHL,TUCKER STATION,RD,RD,,,-85.528419,38.200360
4,431949996,I 64 EAST,I 265 RAMP,,,,,-85.504954,38.222603
5,593499996,I 265 NORTH,I 265 RAMP,,,,,-85.505546,38.222127


In [10]:
MST = df.main_street
XST = df.cross_street

streets = pd.concat((MST, XST)).unique()

In [12]:

# Some street names start with digits


pattern = '([^0-9]*)([0-9]+)([^0-9]*)'
digit = '[0-9]'

def searcher(pattern):
    def do(value):
        match = re.search(pattern, value)
        return bool(match)
    return do

hasdigit = searcher(digit)
    
NST = {street for street in streets if hasdigit(street)}
NST = pd.DataFrame(list(NST), columns=['name'])

NST['hasdigit'] = NST.name.apply(searcher('[0-9]'))
NST['firstdigit'] = NST.name.apply(searcher('^[0-9]'))
NST

,name,hasdigit,firstdigit
0,HIGHWAY 148,True,False
1,18TH,True,True
2,I 265 SOUTH,True,False
3,27TH,True,True
4,16TH,True,True
...,...,...,...
80,1ST,True,True
81,HIGHWAY 22,True,False
82,39TH,True,True
83,9TH STREET,True,True


In [37]:

#prefixes]
starts = NST.name.str.get(0).unique()
prefixes = [char for char in starts if not str.isnumeric(char)]
match = lambda v:any(v.startswith(prefix) for prefix in prefixes)
matches = pd.DataFrame(NST[NST.name.apply(match)].name)
matches['start'] = NST.name.str.get(0)
display(matches.groupby('start')['name'].unique())


start
H    [HIGHWAY 148, HIGHWAY 329, HIGHWAY 42, HIGHWAY...
I    [I 265 SOUTH, I 264 EAST, I 64 WEST, IN-265 NO...
K    [KFEC GATE6, KY-841, KY 841, KFEC GATE2, KFEC ...
S                       [SYLVANIA NO 4, SYLVANIA NO 6]
U                                     [U S HIGHWAY 42]
Name: name, dtype: object

In [70]:
matches.name = matches.name.str.replace('-', ' ')

def get_prefix(value:str):
    first, *junk = value.split()
    return first

matches['prefix'] = matches.name.apply(get_prefix)

matches.groupby('prefix').name.unique()

matches.update(matches[matches.name.str.startswith('U')].name.str.replace("U S", "US"))
prefixes = matches.groupby('prefix').name.unique().index.to_list()



In [71]:
prefixes

['HIGHWAY', 'I', 'IN', 'KFEC', 'KY', 'SYLVANIA', 'US']

In [ ]:


name_expr  = pyp.Word(pyp.nums) + pyp.ZeroOrMore(pyp.Word(pyp.printables))
name_expr.parse_string('9TH And Fk').as_list()


In [ ]:
x, *y = [name_expr.parse_string(name) for name in filtered][14]
x,y

('22', ['ND', 'ST', 'CONNECTOR'])

In [ ]:

def do(name):
    number, *coda = name_expr.parse_string(name)
    C1, *C2 = coda
    return number, C1, C2

out = pd.DataFrame.from_records([do(name) for name in filtered],
                          columns = 'number C1 coda'.split())
out.number = out.number.astype(int)
out = out.sort_values('number')
out.head()
out.coda

def do2(value):
    if value == list():
        return pd.NA
    elif value == ['STREET']:
        return pd.NA
    else:
        return ' '.join(value)
    
out.coda = out.coda.apply(do2)
out = out.replace({"ST CONNECTOR":"STREET CONNECTOR"})
out


,number,C1,coda
10,1,ST,<NA>
22,2,ND,<NA>
34,3,RD,<NA>
33,3,RD,<NA>
43,4,TH,<NA>
44,5,TH,<NA>
45,6,TH,<NA>
47,7,TH,<NA>
46,7,TH,<NA>
48,8,TH,<NA>
